# AirQo Low-Cost Air Quality Monitor Calibration Challenge

https://zindi.africa/competitions/airqo-low-cost-air-quality-monitor-calibration-challenge

Бейзлайн для соревнования по предсказанию качества воздуха на платформе Zindi

In [37]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [11]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

subm = pd.read_csv('SampleSubmission.csv')

In [12]:
train.head()

,ID,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road,ref_pm2_5
0,ID_0038MG0B,2020-04-23 17:00:00+03:00,USEmbassy,6.819048,7.313810,6.794048,7.838333,0.807417,22.383333,0.299255,32.592686,1199,4374,21,14,6834,130,25.0
1,ID_008ASVDD,2020-02-23 19:00:00+03:00,USEmbassy,57.456047,67.883488,55.643488,70.646977,0.712417,25.350000,0.299255,32.592686,1199,4374,21,14,6834,130,68.0
2,ID_009ACJQ9,2021-01-23 04:00:00+03:00,Nakawa,170.009773,191.153636,165.308636,191.471591,0.907833,20.616667,0.331740,32.609510,1191,5865,31,-11,4780,500,149.7
3,ID_00IGMAQ2,2019-12-04 09:00:00+03:00,USEmbassy,49.732821,61.512564,0.000000,0.000000,0.949667,21.216667,0.299255,32.592686,1199,4374,21,14,6834,130,54.0
4,ID_00P76VAQ,2019-10-01 01:00:00+03:00,USEmbassy,41.630455,51.044545,41.725000,51.141364,0.913833,18.908333,0.299255,32.592686,1199,4374,21,14,6834,130,39.0


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10932 entries, 0 to 10931
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               10932 non-null  object 
 1   created_at       10932 non-null  object 
 2   site             10932 non-null  object 
 3   pm2_5            10932 non-null  float64
 4   pm10             10932 non-null  float64
 5   s2_pm2_5         10932 non-null  float64
 6   s2_pm10          10932 non-null  float64
 7   humidity         10926 non-null  float64
 8   temp             10915 non-null  float64
 9   lat              10932 non-null  float64
 10  long             10932 non-null  float64
 11  altitude         10932 non-null  int64  
 12  greenness        10932 non-null  int64  
 13  landform_90m     10932 non-null  int64  
 14  landform_270m    10932 non-null  int64  
 15  population       10932 non-null  int64  
 16  dist_major_road  10932 non-null  int64  
 17  ref_pm2_5   

In [14]:
test.head()

,ID,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road
0,ID_00OZLF7X,2020-03-13 07:00:00+03:00,USEmbassy,31.900455,35.515455,31.672273,37.051818,0.927167,21.175000,0.299255,32.592686,1199,4374,21,14,6834,130
1,ID_00ZI0D98,2020-08-08 10:00:00+03:00,Makerere,53.581818,66.603636,50.586364,64.651818,0.811583,22.350000,0.333501,32.568561,1233,6340,21,28,8518,475
2,ID_017GTLAU,2020-08-25 09:00:00+03:00,Makerere,62.377500,71.647500,59.023333,69.766667,0.902000,20.766667,0.333501,32.568561,1233,6340,21,28,8518,475
3,ID_01IBM7T2,2020-06-15 16:00:00+03:00,USEmbassy,33.310294,36.958824,33.060882,38.674412,0.643417,25.483333,0.299255,32.592686,1199,4374,21,14,6834,130
4,ID_01II27D4,2021-01-13 00:00:00+03:00,Nakawa,64.782045,75.247500,64.638182,77.108864,0.939667,20.133333,0.331740,32.609510,1191,5865,31,-11,4780,500


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2733 entries, 0 to 2732
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2733 non-null   object 
 1   created_at       2733 non-null   object 
 2   site             2733 non-null   object 
 3   pm2_5            2733 non-null   float64
 4   pm10             2733 non-null   float64
 5   s2_pm2_5         2733 non-null   float64
 6   s2_pm10          2733 non-null   float64
 7   humidity         2732 non-null   float64
 8   temp             2729 non-null   float64
 9   lat              2733 non-null   float64
 10  long             2733 non-null   float64
 11  altitude         2733 non-null   int64  
 12  greenness        2733 non-null   int64  
 13  landform_90m     2733 non-null   int64  
 14  landform_270m    2733 non-null   int64  
 15  population       2733 non-null   int64  
 16  dist_major_road  2733 non-null   int64  
dtypes: float64(8),

In [16]:
subm.head()

,ID,ref_pm2_5
0,ID_00OZLF7X,NaN
1,ID_00ZI0D98,NaN
2,ID_017GTLAU,NaN
3,ID_01IBM7T2,NaN
4,ID_01II27D4,NaN


**const baseline**

In [18]:
sub = pd.DataFrame()
sub['ID'] = test.iloc[:,0]
sub['ref_pm2_5'] = train['ref_pm2_5'].mean()
sub.to_csv('const.csv', index=False)

Объединим данные и запомним кол-во записей в train

In [19]:
idx_train = train.shape[0] 

all_data = pd.concat((train, test)).reset_index(drop=True)

Для бейзлайна возьмем только числовые признаки. Заполним пропуски средним

In [27]:
num_cols = all_data.drop(['ref_pm2_5'], axis=1).select_dtypes(exclude=['object']).columns

In [29]:
for col in num_cols:
    all_data[col] = all_data[col].fillna(all_data[col].mean())
all_data[num_cols].isna().sum()    

pm2_5              0
pm10               0
s2_pm2_5           0
s2_pm10            0
humidity           0
temp               0
lat                0
long               0
altitude           0
greenness          0
landform_90m       0
landform_270m      0
population         0
dist_major_road    0
dtype: int64

In [34]:
train_df = all_data[:idx_train]
test_df = all_data[idx_train:]
X = train_df[num_cols]
y = train_df.ref_pm2_5

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)


In [38]:
model = Ridge(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f'R^2 score on the X_test is: {mean_squared_error(y_test, y_pred)}')

R^2 score on the X_test is: 287.51055543670276


In [43]:
model = RandomForestRegressor(random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f'R^2 score on the X_test is: {mean_squared_error(y_test, y_pred)}')

R^2 score on the X_test is: 227.3128847301829


In [44]:
test_df = test_df[num_cols]
preds = model.predict(test_df)

In [45]:
sub = pd.DataFrame()
sub['ID'] = test.iloc[:,0]
sub['ref_pm2_5'] = preds
sub.to_csv('rf.csv', index=False)